<a href="https://colab.research.google.com/github/mmistroni/Magentic-AlgoTrading101/blob/main/openbbagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### What we will attempt to  do here is to create an Agent that will call Openbb functions and big query
### We will use Gemini as Smolagents inference model runs out of token

# Importing Libraries

In [1]:
!pip install litellm
!pip install openbb
!pip install google-api-python-client
!pip install fsspec
!pip install gcsfs
!pip install smolagents -U



# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key
from google.colab import auth
auth.authenticate_user()


In [2]:
from openbb import obb
from google.colab import userdata
obb.equity.price.historical(symbol='AAPL',provider='yfinance')


Extensions to add: sec@1.4.5
Extensions to remove: sec@1.4.4

Building...


OBBject[T]

id: 068cb110-45b6-7ce7-8000-f5fb373c1ab7
results: [{'date': datetime.date(2024, 9, 17), 'open': 215.75, 'high': 216.89999389...
provider: yfinance
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'}, 's...

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

In [7]:
#import requests

import pandas as pd
import numpy as np
from datetime import date, datetime
!mkdir -p data
from apiclient.http import MediaIoBaseDownload
from pandas.tseries.offsets import BDay
import numpy as np
from datetime import datetime, date
from pandas.tseries.offsets import BDay
from google.cloud import storage
from googleapiclient.discovery import build
from datetime import date
from google.cloud import bigquery
from google.cloud import bigquery_storage



In [10]:


def read_from_bucket(filename, bucket_name, to_panda=True):
  gcs_service = build('storage', 'v1')
  # The name for the new bucket
  holder = 'data/{}'.format(filename.split('/')[-1])
  print('Reading {} from {}'.format(filename,bucket_name))
  with open(holder, 'wb') as f:
    request = gcs_service.objects().get_media(bucket=bucket_name,
                                              object=filename)
    media = MediaIoBaseDownload(f, request)

    done = False
    while not done:
      # _ is a placeholder for a progress object that we ignore.
      # (Our file is small, so we skip reporting progress.)
      _, done = media.next_chunk()

  print('Download complete')
  if to_panda:
    return pd.read_csv(holder)



# Construct a BigQuery client object.

def get_bigquery_as_dataframe(sql_stmnt):
  client = bigquery.Client(project='datascience-projects')
  bqstorageclient = bigquery_storage.BigQueryReadClient()
  query_job = client.query(sql_stmnt)
  return query_job.result().to_dataframe()


def get_senate_disclosures_obb(start_date:str = '2025-01-01',
                                       end_date:str = '2025-03-31')-> dict:
  sql = f"""SELECT
          DISTINCT AS_OF_DATE,
          REPLACE(TICKER, 'Ticker:', '') AS TICKER
        FROM
          `datascience-projects`.`gcp_shareloader`.`senate_disclosures`
        WHERE
          DISCLOSURE LIKE '%Purchase%'
          AND AS_OF_DATE BETWEEN DATE '{start_date}'
          AND DATE '{end_date}' /* Replace with your desired Date A and Date B */
        ORDER BY
          AS_OF_DATE;
        """



  bqclient = bigquery.Client(project='datascience-projects')
  dataframe = (
    bqclient.query(sql)
    .result()
    .to_dataframe()
  )

  dataframe['date'] = pd.to_datetime(dataframe['AS_OF_DATE'])
  # Format the timestamp to a clear date string
  dataframe['date'] = dataframe['date'].dt.strftime('%Y-%m-%d')
  # Convert the DataFrame to a JSON string
  # The 'orient' parameter is crucial for the desired JSON structure.
  # 'records' is a common and useful format for APIs.
  json_data = dataframe.to_json(orient='records')

  return json_data



# Compare vs industry. Check metrics against every other company  until we can find industry / sector metrics

#get_senate_disclosures_obb()



### Smolagent tools

In [11]:
from smolagents import CodeAgent, tool
from google.colab import userdata
from smolagents import CodeAgent, OpenAIServerModel

# Replace with your actual Gemini API Key
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

model = OpenAIServerModel(
    model_id="gemini-2.5-flash",  # You can choose a different Gemini model
    api_base="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=GEMINI_API_KEY,
)

@tool
def get_senate_disclosures(start_date:str, end_date:str) -> dict:
  """
    Return senate disclosures between a start_date and an end_date.
    Args:
        start_date(str): a string representing a date in the format YYYY-MM-DD
        end_date(str): a string representing a date in the format YYYY-MM-DD
  """
  return get_senate_disclosures_obb(start_date, end_date)


@tool
def get_historical_prices_for_ticker(ticker:str,
                                     start_date:str,
                                     end_date:str) -> dict:
  """
    Return historical prices for a ticker.
    Args:
        ticker (str): a string representing a ticker
        start_date(str): a string representing a date in the format YYYY-MM-DD
        end_date(str): a string representing a date in the format YYYY-MM-DD
  """

  return obb.equity.price.historical(symbol='AAPL', start_date=start_date,
         end_date=end_date, provider='yfinance')

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

#open bb




In [15]:

input_msg = """I want you to fetch all senate disclosures between 1st of March 2025 and 1st of May 2025.2. find how many dates we have for each ticker
3. Select the top 20 tickers which have the higher number of occurrences 4. present them in a table with two columns, Ticker and Occurrences"""
# Your system message
system_message = """You are a financial analyst agent.
Your goal is to help extract insight from senate disclosures, which represents shares  bought by
US Senate members, and historical prices of those stocks.
By analyzing this data you must estabilish if there is a relation between shares bought
by US senate and their prices.
Use the tools availabale at your disposal for this
"""
agent = CodeAgent(tools=[suggest_menu, get_historical_prices_for_ticker, get_senate_disclosures],
                  model=model, instructions=system_message)

agent.run(input_msg)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I want you to fetch all senate disclosures between 1st of March 2025 and 1st of May 2025.2. find how many dates │
│ we have for each ticker                                                                                         │
│ 3. Select the top 20 tickers which have the higher number of occurrences 4. present them in a table with two    │
│ columns, Ticker and Occurrences                                                                                 │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.5-flash ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  disclosures = get_senate_disclosures(start_date="2025-03-01", end_date="2025-05-01")                             
  print(disclosures)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
[{"AS_OF_DATE":1741046400000,"TICKER":"ET","date":"2025-03-04"},{"AS_OF_DATE":1741046400000,"TICKER":"ARLP","date":
"2025-03-04"},{"AS_OF_DATE":1741219200000,"TICKER":"CRWD","date":"2025-03-06"},{"AS_OF_DATE":1741219200000,"TICKER"
:"AMP","date":"2025-03-06"},{"AS_OF_DATE":1741219200000,"TICKER":"AMZN","date":"2025-03-06"},{"AS_OF_DATE":17412192
00000,"TICKER":"","date":"2025-03-06"},{"AS_OF_DATE":1741305600000,"TICKER":"EPAC","date":"2025-03-07"},{"AS_OF_DAT
E":1741305600000,"TICKER":"SQ","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"WYNN","date":"2025-03-07"
},{"AS_OF_DATE":1741305600000,"TICKER":"FI","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"BABA","date"
:"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"PFE","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER"
:"MSFT","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"ABT","date":"2025-03-07"},{"AS_OF_DATE":17413056
00000,"TICKER":"SHOP","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"MRK","date":"2025-03-07"},{"AS_OF_
DATE":1741305600000,"TICKER":"DSCSY","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"TSLA","date":"2025-
03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"V","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"WMT","d
ate":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"DG","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICK
ER":"WTW","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"SNOW","date":"2025-03-07"},{"AS_OF_DATE":17413
05600000,"TICKER":"AVGO","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"SBUX","date":"2025-03-07"},{"AS
_OF_DATE":1741305600000,"TICKER":"TWLO","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"UBER","date":"20
25-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"STRL","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"H
UM","date":"2025-03-07"},{"AS_OF_DATE":1741305600000,"TICKER":"UNP","date":"2025-03-07"},{"AS_OF_DATE":174147840000
0,"TICKER":"AMGN","date":"2025-03-09"},{"AS_OF_DATE":1741478400000,"TICKER":"PLD","date":"2025-03-09"},{"AS_OF_DATE
":1741478400000,"TICKER":"CI","date":"2025-03-09"},{"AS_OF_DATE":1741478400000,"TICKER":"OXY","date":"2025-03-09"},
{"AS_OF_DATE":1741478400000,"TICKER":"NVDA","date":"2025-03-09"},{"AS_OF_DATE":1741478400000,"TICKER":"HD","date":"
2025-03-09"},{"AS_OF_DATE":1741478400000,"TICKER":"QCOM","date":"2025-03-09"},{"AS_OF_DATE":1741478400000,"TICKER":
"SHW","date":"2025-03-09"},{"AS_OF_DATE":1741478400000,"TICKER":"BUSE","date":"2025-03-09"},{"AS_OF_DATE":174147840
0000,"TICKER":"ORCL","date":"2025-03-09"},{"AS_OF_DATE":1741478400000,"TICKER":"APP","date":"2025-03-09"},{"AS_OF_D
ATE":1741478400000,"TICKER":"PHM","date":"2025-03-09"},{"AS_OF_DATE":1741478400000,"TICKER":"BC","date":"2025-03-09
"},{"AS_OF_DATE":1741478400000,"TICKER":"NVT","date":"2025-03-09"},{"AS_OF_DATE":1741478400000,"TICKER":"PFE","date
":"2025-03-09"},{"AS_OF_DATE":1741564800000,"TICKER":"HD","date":"2025-03-10"},{"AS_OF_DATE":1741564800000,"TICKER"
:"MSTR","date":"2025-03-10"},{"AS_OF_DATE":1741564800000,"TICKER":"AVGO","date":"2025-03-10"},{"AS_OF_DATE":1741564
800000,"TICKER":"OXY","date":"2025-03-10"},{"AS_OF_DATE":1741564800000,"TICKER":"ORCL","date":"2025-03-10"},{"AS_OF
_DATE":1741564800000,"TICKER":"ELV","date":"2025-03-10"},{"AS_OF_DATE":1741564800000,"TICKER":"MTSI","date":"2025-0
3-10"},{"AS_OF_DATE":1741564800000,"TICKER":"SNOW","date":"2025-03-10"},{"AS_OF_DATE":1741564800000,"TICKER":"CSGP"
,"date":"2025-03-10"},{"AS_OF_DATE":1741564800000,"TICKER":"UNP","date":"2025-03-10"},{"AS_OF_DATE":1741564800000,"
TICKER":"QCOM","date":"2025-03-10"},{"AS_OF_DATE":1741564800000,"TICKER":"NVDA","date":"2025-03-10"},{"AS_OF_DATE":
1741564800000,"TICKER":"FIS","date":"2025-03-10"},{"AS_OF_DATE":1741564800000,"TICKER":"SHW","date":"2025-03-10"},{
"AS_OF_DATE":1741564800000,"TICKER":"GS","date":"2025-03-10"},{"AS_OF_DATE":1741564800000,"TICKER":"RTX","date":"20
25-03-10"},{"AS_OF_DATE":1741564800000,"

[Step 1: Duration 6.25 seconds| Input tokens: 2,480 | Output tokens: 49]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  from collections import Counter                                                                                  
                                                                                                                   
  ticker_counts = Counter()                                                                                        
                                                                                                                   
  # Assuming 'disclosures' is a list of dictionaries as initially returned                                         
  for disclosure_item in disclosures:                                                                              
      # Ensure each item is a dictionary before trying to access 'TICKER'                                          
      if isinstance(disclosure_item, dict):                                                                        
          ticker = disclosure_item.get("TICKER")                                                                   
          # Ensure ticker is a non-empty string and not '$SKIUSD'                                                  
          if isinstance(ticker, str) and ticker.strip() and ticker != "$SKIUSD":                                   
              ticker_counts[ticker] += 1                                                                           
                                                                                                                   
  # Get the top 20 tickers by occurrence                                                                           
  top_20_tickers = ticker_counts.most_common(20)                                                                   
                                                                                                                   
  # Prepare the output table                                                                                       
  table_headers = ["Ticker", "Occurrences"]                                                                        
  table_rows = []                                                                                                  
  for ticker, count in top_20_tickers:                                                                             
      table_rows.append([ticker, count])                                                                           
                                                                                                                   
  # Format as a string table                                                                                       
  header_line = "|".join(table_headers)                                                                            
  separator_line = "|".join(["-" * len(h) for h in table_headers])                                                 
  data_lines = ["|".join(map(str, row)) for row in table_rows]                                                     
                                                                                                                   
  final_table = "\n".join([header_line, separator_line] + data_lines)                                              
  final_answer(final_table)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Ticker|Occurrences
------|-----------

[Step 4: Duration 9.57 seconds| Input tokens: 64,789 | Output tokens: 1,484]

'Ticker|Occurrences\n------|-----------'

In [ ]:
from litellm import completion
from typing import List, Dict


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


what_to_help_with = input("What do you need help with?")

messages = [
    {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
    {"role": "user", "content": what_to_help_with}
]

response = generate_response(messages)
print(response)